# Uber Data

In [56]:
import os
import ast
import glob
import numpy as np
import pandas as pd
from collections import Counter
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [57]:
X = pd.read_csv('X.csv')
polygon_points = pd.read_json('Data/uber/540_hexclusters.json',orient='values')
X.columns
X = X[0:100]

Index(['Personal or Business', 'Day of Month', 'Placement - Time',
       'Confirmation - Time', 'Arrival at Pickup - Time', 'Pickup - Time',
       'Distance (KM)', 'Temperature', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'Platform Type 1', 'Platform Type 2',
       'Platform Type 3', 'Platform Type 4', 'Bearing', 'Monday', 'Tuesday',
       'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
      dtype='object')

In [59]:
def point_in_polygon(data, polygons_df):
    out_pickup = list()
    for n,coord in enumerate(data['Pickup Long']):
        list_movid = list()
        for k,pol in enumerate(polygons_df['features']):
            mov_id = pol['properties']['MOVEMENT_ID']
            poly = pol['geometry']['coordinates'][0]
            polygon = Polygon(poly)
            point = Point(coord,data.loc[n,'Pickup Lat'])
            if polygon.contains(point) and point.within(polygon):
                list_movid.append(mov_id)
        out_pickup.append(list_movid)
    out_dest = list()
    for n,coord in enumerate(data['Destination Long']):
        list_movid = list()
        for k,pol in enumerate(polygons_df['features']):
            mov_id = pol['properties']['MOVEMENT_ID']
            poly = pol['geometry']['coordinates'][0]
            polygon = Polygon(poly)
            point = Point(coord,data.loc[n,'Destination Lat'])
            if polygon.contains(point) and point.within(polygon):
                list_movid.append(mov_id)
        out_dest.append(list_movid)
    return out_pickup, out_dest


def cleanup_lists(lst):
    out = list()
    for n,v in enumerate(lst):
        if len(v) == 0:
            out.append(-999)

        elif len(v) == 1:
            out.append(int(v[0]))
        else:
            out.append(-888)
    return out

In [60]:
pickup1, dest1 = point_in_polygon(data=X, polygons_df=polygon_points)
print(Counter([len(lst) for lst in dest1]))
print(Counter([len(lst) for lst in pickup1]))
dest1 = cleanup_lists(lst=dest1)
pickup1 = cleanup_lists(lst=pickup1)

Counter({1: 100})
Counter({1: 100})
